In [1]:
!pip install geopy

import numpy as np
import pandas as pd
import tensorflow as tf
from geopy import distance
import geopandas
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.utils import to_categorical

import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

error_metric_used = 'mean_squared_error'
optimizer = tf.optimizers.Adam()

BASE_EPOCHS = 100

### Load Data

In [ ]:
# Load data here
df = pd.DataFrame()
df.head()

### Prepare Data

In [ ]:
#Calculate distance from customer home to merchant location
df['home_loc'] = geopandas.points_from_xy(df['long'], df['lat'])
df['merch_loc'] = geopandas.points_from_xy(df['merch_long'], df['merch_lat'])

df['merch_distance'] = df.apply(lambda row: row['home_loc'].distance(row['merch_loc']), axis=1)

df.drop(columns=['lat', 'long', 'merch_lat', 'merch_long', 'home_loc', 'merch_loc'], inplace=True)

# Calculate age of customer from trasnaction time and date of birth
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

df['user_age'] = (df['trans_date_trans_time'] - df['dob']).dt.days
df.drop(columns=['dob'], inplace=True)

# Keep only the time of the transaction
df['time'] = df['trans_date_trans_time'].dt.time
# Change time to minutes
df['time'] = df['time'].apply(lambda x: x.hour * 60 + x.minute)

df.drop(columns=['trans_date_trans_time'], inplace=True)

In [ ]:
# Remove unnecessary columns
cleaned_df = df.copy()
cleaned_df.drop(columns=['cc_num', 'merchant', 'first',	'last', 'street','city','zip', 'state','job', 'trans_num', 'unix_time'], inplace=True)

# Scale the numerical data
scaler = StandardScaler()
num = ['amt', 'city_pop', 'merch_distance', 'user_age', 'time']
scaler.fit(cleaned_df[num])
cleaned_df[num] = scaler.transform(cleaned_df[num])

# One hot encoding on categorical data
new_df = pd.get_dummies(cleaned_df)

In [3]:
model = keras.models.load_model('my_model.keras')

c:\Users\desla\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


### Predictions using trained model

In [ ]:
def pred_model(df, model):
  y = df["is_fraud"]
  X = df.drop(columns={"is_fraud"})

  pred = model.predict(X)
  pred_labels = np.argmax(pred, axis=1)

  conf_matrix = confusion_matrix(y, pred_labels)

  print(conf_matrix)
  print(classification_report(y, pred_labels))

In [ ]:
pred_model(new_df, model)

### Explanation
##### In terms of data preparation, for the location data I calculated the distance from the customer's home to the purchase location using geopandas. I also used the customer's age along with the transaction time to calculate their age at the time of purchase. Any columns that were either categorical and did not have any information that was relevant, such as address information, or numerical but not important to the situation, like the credit card number, were removed. Columns that were categorical and would not have too many categories for one hot encoding were left in, while the numerical data was scaled.

##### The model design involved six layers, one normalization layer, one dense and one dropout layer, then two dense layers with L1 and L2 regularization to reduce overfitting, and finally one dense output layer. An attempt was made to adjust class weights to account for unbalanced data, but I encountered issues when running the model and I could not be resolves it in time to produce a working model.